<a href="https://colab.research.google.com/github/HajarahM/Mistral-7B/blob/main/Chat_with_MultiplePDFs_Mistral_7B_Instruct1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install langchain
%pip install torch
%pip install sentence_transformers
%pip install faiss-cpu
%pip install huggingface-hub
%pip install pypdf
%pip -q install accelerate
%pip install llama-cpp-python
%pip -q install git+https://github.com/huggingface/transformers


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [92]:
from langchain.chains import RetrievalQA
from langchain.embeddings import OllamaEmbeddings
from langchain.llms import Ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFium2Loader
import ocrmypdf

In [95]:
#load pdf files
loader = DirectoryLoader("./Laws/Petroleum")
data = loader.load()

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [82]:
print(data)

In [69]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10, separators=['\n\n', '\n', '.'])

text_chunks = text_splitter.split_documents(data)


In [70]:
len(text_chunks)

0

In [72]:
#get the third chunk
text_chunks[10]

IndexError: list index out of range

In [53]:
#Step 06:Downlaod the Embeddings
embeddings = OllamaEmbeddings(model="mistral")

In [54]:
from langchain.vectorstores import Chroma
vector_store = Chroma.from_documents(text_chunks, embedding=embeddings)
print('saved embeddings to vector_store')

saved embeddings to vector_store


In [12]:
#Step 08: Create Embeddings for each of the Text Chunk
import os
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)
print('saving embeddings to vector_store')
folder_path="FAISS_vector_store"
if os.path.exists(folder_path):
    faiss_index=FAISS.load_local(folder_path, embeddings)
    faiss_index.merge_from(vector_store)
    faiss_index.save_local(folder_path)
else:
    vector_store.save_local(folder_path)

KeyboardInterrupt: 

In [56]:
#Import Model
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = Ollama(
    base_url="http://localhost:11434",
    model = "mistral",
    verbose=True,
    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
    )

In [57]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 1}))

In [60]:
query = "in UNCITRAL model law, who determines the number of arbitrators"

In [61]:
qa.run(query)


As per the UNCITRAL Model Law, the number of arbitrators is determined by the parties involved. This means that the parties will agree on the number of arbitrators before commencing any arbitration proceedings.

'\nAs per the UNCITRAL Model Law, the number of arbitrators is determined by the parties involved. This means that the parties will agree on the number of arbitrators before commencing any arbitration proceedings.'

In [32]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")


Under Section 28 of the Arbitration Act, the rules applicable to the substance of the dispute would still be in effect. The decision-making process by the panel of arbitrators would also continue, but if a party dies during the proceedings, their rights and obligations under the arbitration agreement would pass on to their estate.

If the award is already made before the death of a party, it would still be binding on the parties involved. However, in some cases, the deceased party's estate may challenge the award or seek a settlement if there are sufficient funds available.

If the award has not yet been made when a party dies, the arbitration proceedings would likely continue, but with the representation of the deceased party's estate. The panel of arbitrators would need to consider any relevant legal issues arising from the death of the party, such as whether the deceased party had previously waived their right to challenge the award or whether the arbitration agreement provides for

SystemExit: 

/opt/miniconda3/envs/data_processor/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
